<a href="https://colab.research.google.com/github/Angel-Diaz-H/scripts-python-sql/blob/main/sql_scripts_respaldos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql" # Nombre del archivo de salida

#leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Diccionario para agrupar condiciones por tabla y columna
condiciones_agrupadas = {} # Formato: {('tabla', 'columna'): [valor1, valor2, ...]}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    # ignorar y saltar comentarios
    if not sentencia or sentencia.startswith("--"):
        continue

    # DELETE
    if sentencia.upper().startswith("DELETE FROM"):
        # Extraer tabla
        tabla_match = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # Extraer la condición WHERE
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion_completa = sentencia[where_idx + len("WHERE"):].strip()

                # Extraer la columna y el valor para la condición IN
                match_in = re.match(r"(\w+)\s*=\s*(.*)", condicion_completa)
                if match_in:
                    columna = match_in.group(1).strip()
                    valor = match_in.group(2).strip()

                    # Agrupar la condición
                    if (tabla, columna) not in condiciones_agrupadas:
                        condiciones_agrupadas[(tabla, columna)] = []
                    # Asegurarse de no agregar valores duplicados si eso es deseado
                    if valor not in condiciones_agrupadas[(tabla, columna)]:
                         condiciones_agrupadas[(tabla, columna)].append(valor)

    # UPDATE (mantener lógica existente si la necesitas, o adaptar para agrupar si aplica)
    elif sentencia.upper().startswith("UPDATE"):
        # search tabla después de UPDATE
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # WHERE (en todo el bloque)
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion = sentencia[where_idx:]
                linea_select = f"SELECT * FROM {tabla} {condicion}"
                # Aquí podrías decidir si quieres agrupar UPDATEs también,
                # o mantenerlos separados como en tu código original.
                # Para este ejemplo, los mantenemos separados y no los incluimos en la salida agrupada de DELETEs.
                # Si quieres incluir SELECTs de UPDATEs, necesitarías otro mecanismo de almacenamiento.


# Generar sentencias DELETE agrupadas (o SELECT, si ese es tu objetivo final)
sentencias_agrupadas_generadas = []
for (tabla, columna), valores in condiciones_agrupadas.items():
    if valores: # Solo generar si hay valores para la condición IN
        #formatear los valores con IN
        valores_formateados = ", ".join(valores)
        #generar select con Where in
        linea_agrupada = f"SELECT * FROM {tabla} WHERE {columna} IN ({valores_formateados});"
        sentencias_agrupadas_generadas.append(linea_agrupada)

#save
with open(archivo_salida, "w") as f:
    for sel in sentencias_agrupadas_generadas:
        f.write(sel + "\n")

print(f"{len(sentencias_agrupadas_generadas)} sentencias respaldadas (agrupadas) en '{archivo_salida}'")

0 sentencias respaldadas (agrupadas) en 'backup_agrupado.sql'


In [ ]:
import re

archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql" # Nombre del archivo de salida

#leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Diccionario para agrupar condiciones por tabla y patrón de WHERE para DELETE
condiciones_agrupadas_delete = {} # Formato: {('tabla', 'patron_where'): [valor1, valor2, ...]}

# Diccionario para agrupar condiciones por tabla y patrón de WHERE para UPDATE
condiciones_agrupadas_update = {} # Formato: {('tabla', 'patron_where'): [valor1, valor2, ...]}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    # ignorar y saltar comentarios
    if not sentencia or sentencia.startswith("--"):
        continue

    # DELETE
    if sentencia.upper().startswith("DELETE FROM"):
        # Extraer tabla
        tabla_match = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # Extraer la condición WHERE
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion_completa = sentencia[where_idx + len("WHERE"):].strip()

                # Intentar extraer columna y valor para agrupar
                match_in = re.match(r"(\w+)\s*=\s*(.*)", condicion_completa)
                if match_in:
                    columna = match_in.group(1).strip()
                    valor = match_in.group(2).strip()
                    # Usamos la tabla y la columna como clave de agrupación
                    clave_agrupacion = (tabla, columna)

                    if clave_agrupacion not in condiciones_agrupadas_delete:
                        condiciones_agrupadas_delete[clave_agrupacion] = []
                    if valor not in condiciones_agrupadas_delete[clave_agrupacion]:
                         condiciones_agrupadas_delete[clave_agrupacion].append(valor)
                # Manejar DELETE con WHERE más complejo si es necesario, aquí se ignoran para la agrupación actual

    # UPDATE
    elif sentencia.upper().startswith("UPDATE"):
        # Extraer tabla después de UPDATE
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # Encontrar la cláusula WHERE
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                # Extraer la cláusula WHERE completa
                condicion_completa = sentencia[where_idx + len("WHERE"):].strip()

                # Intentar agrupar UPDATEs con cláusulas WHERE similares
                # Dividir la condición WHERE por AND
                condiciones_and = re.split(r"\s+AND\s+", condicion_completa, re.IGNORECASE)

                # Si hay al menos dos condiciones y la última tiene formato 'columna = valor'
                if len(condiciones_and) > 0:
                    ultima_condicion = condiciones_and[-1].strip()
                    match_ultima_condicion = re.match(r"(\w+)\s*=\s*(.*)", ultima_condicion)

                    if match_ultima_condicion:
                        columna_agrupacion = match_ultima_condicion.group(1).strip()
                        valor_agrupacion = match_ultima_condicion.group(2).strip()

                        # Construir el patrón de WHERE para la agrupación (sin la última condición)
                        patron_where = " AND ".join(condiciones_and[:-1]).strip()
                        if not patron_where: # Si solo hay una condición
                            patron_where = columna_agrupacion # Usamos solo la columna

                        clave_agrupacion = (tabla, patron_where)

                        if clave_agrupacion not in condiciones_agrupadas_update:
                            condiciones_agrupadas_update[clave_agrupacion] = []
                        if valor_agrupacion not in condiciones_agrupadas_update[clave_agrupacion]:
                            condiciones_agrupadas_update[clave_agrupacion].append(valor_agrupacion)
                    else:
                         # Si la última condición no tiene el formato 'columna = valor', se maneja individualmente
                         linea_select = f"SELECT * FROM {tabla} WHERE {condicion_completa};"
                         # Aquí podrías agregarla a una lista separada si no quieres agruparla


# Generar sentencias SELECT agrupadas para DELETE
sentencias_select_delete_agrupadas = []
for (tabla, columna), valores in condiciones_agrupadas_delete.items():
    if valores: # Solo generar si hay valores para la condición IN
        valores_formateados = ", ".join(valores)
        linea_agrupada = f"SELECT * FROM {tabla} WHERE {columna} IN ({valores_formateados});"
        sentencias_select_delete_agrupadas.append(linea_agrupada)

# Generar sentencias SELECT agrupadas para UPDATE
sentencias_select_update_agrupadas = []
for (tabla, patron_where), valores in condiciones_agrupadas_update.items():
     if valores: # Solo generar si hay valores para la condición IN
         valores_formateados = ", ".join(valores)
         # Si el patrón de WHERE es solo la columna, generamos "WHERE columna IN (...)"
         if patron_where in [item[1] for item in condiciones_agrupadas_update.keys() if item[0] == tabla]:
              linea_agrupada = f"SELECT * FROM {tabla} WHERE {patron_where} IN ({valores_formateados});"
         else:
             # Si hay otras condiciones, generamos "WHERE patron_where AND columna IN (...)"
             # Intentamos extraer la columna de agrupación del patrón para reconstruir la WHERE
             match_columna_patron = re.match(r"(\w+)", patron_where)
             if match_columna_patron:
                  columna_agrupacion = match_columna_patron.group(1)
                  linea_agrupada = f"SELECT * FROM {tabla} WHERE {patron_where} IN ({valores_formateados});"
             else:
                 # Caso de respaldo si no se pudo extraer la columna del patrón
                 # Esto podría ocurrir si el patrón de WHERE es más complejo
                 linea_agrupada = f"SELECT * FROM {tabla} WHERE {patron_where} IN ({valores_formateados});"

         sentencias_select_update_agrupadas.append(linea_agrupada)


# Unir las sentencias SELECT generadas de DELETE y UPDATE
todas_sentencias_select = sentencias_select_delete_agrupadas + sentencias_select_update_agrupadas

#save
with open(archivo_salida, "w") as f:
    for sel in todas_sentencias_select:
        f.write(sel + "\n")

print(f"{len(todas_sentencias_select)} sentencias respaldadas (SELECT) en '{archivo_salida}'")

17 sentencias respaldadas (SELECT) en 'backup_agrupado.sql'


In [ ]:
import re
from collections import defaultdict

archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# Leer el archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir por sentencias que terminan con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Diccionario para agrupar condiciones
# Formato: {(tabla, parte_fija): [(campo_variable, valor)]}
updates_agrupados = defaultdict(list)

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    if sentencia.upper().startswith("UPDATE"):
        # Extraer tabla
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)

            # Extraer la condición completa después de WHERE
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condiciones = sentencia[where_idx + len("WHERE"):].strip()
                # Separar las condiciones por AND
                condiciones_lista = [cond.strip() for cond in condiciones.split("AND")]

                # Detectar la condición variable (que cambia en cada update)
                # Para este ejemplo, asumimos que la variable es la que tiene valor tipo string o es la última condición
                # Si quieres algo más robusto, puedes adaptar esta parte.
                parte_fija = []
                campo_variable = None
                valor_variable = None

                for cond in condiciones_lista:
                    if "'" in cond or '"' in cond:  # si es string (o podría ser la variable)
                        campo_valor = re.match(r"(\w+)\s*=\s*(['\"].*?['\"])", cond)
                        if campo_valor:
                            campo_variable = campo_valor.group(1).strip()
                            valor_variable = campo_valor.group(2).strip()
                        else:
                            parte_fija.append(cond)
                    else:
                        parte_fija.append(cond)

                # Convertir parte_fija a un string ordenado para que se agrupen bien
                parte_fija_str = " AND ".join(parte_fija)

                if campo_variable and valor_variable:
                    updates_agrupados[(tabla, parte_fija_str, campo_variable)].append(valor_variable)
                else:
                    # Caso donde no hay campo variable; respaldar todo
                    updates_agrupados[(tabla, parte_fija_str, "")].append("")  # Valor vacío

# Generar las líneas de respaldo
with open(archivo_salida, "w") as f:
    for (tabla, parte_fija, campo_variable), valores in updates_agrupados.items():
        if campo_variable and valores:
            valores_unicos = sorted(set(valores))  # sin duplicados
            valores_in = ", ".join(valores_unicos)
            linea_select = f"SELECT * FROM {tabla} WHERE {parte_fija} AND {campo_variable} IN ({valores_in});"
        else:
            # Caso donde no se detectó variable (puede ser un WHERE fijo sin cambios)
            linea_select = f"SELECT * FROM {tabla} WHERE {parte_fija};"

        f.write(linea_select + "\n")

print(f"{len(updates_agrupados)} grupos de UPDATE respaldados en '{archivo_salida}' 🎉✅")


17 grupos de UPDATE respaldados en 'backup_agrupado.sql' 🎉✅


In [ ]:
import re
from collections import defaultdict

archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# Leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Diccionario para agrupar por (tabla, set, condiciones fijas)
# Formato: { (tabla, set, condiciones_fijas) : [CLA_TRAB1, CLA_TRAB2, ...] }
agrupaciones = defaultdict(list)

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    if sentencia.upper().startswith("UPDATE"):
        # Extraer tabla
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        tabla = tabla_match.group(1).strip() if tabla_match else "UNKNOWN"

        # Extraer SET
        set_match = re.search(r"SET\s+(.*?)\s+WHERE", sentencia, re.IGNORECASE | re.DOTALL)
        set_part = set_match.group(1).strip() if set_match else "UNKNOWN"

        # Extraer condiciones WHERE completas
        where_match = re.search(r"WHERE\s+(.*)", sentencia, re.IGNORECASE | re.DOTALL)
        where_part = where_match.group(1).strip() if where_match else ""

        # Separar condiciones AND
        condiciones = [cond.strip() for cond in re.split(r"\s+AND\s+", where_part, flags=re.IGNORECASE)]

        # Encontrar la condición de CLA_TRAB
        cla_trab = None
        condiciones_fijas = []
        for cond in condiciones:
            if cond.upper().startswith("CLA_TRAB"):
                # Extraer valor
                valor_match = re.match(r"CLA_TRAB\s*=\s*(\d+)", cond, re.IGNORECASE)
                if valor_match:
                    cla_trab = valor_match.group(1)
            else:
                condiciones_fijas.append(cond)

        # Usar tuple como clave
        clave = (tabla, set_part, " AND ".join(condiciones_fijas))
        if cla_trab:
            agrupaciones[clave].append(cla_trab)

# Generar SELECTs agrupados
with open(archivo_salida, "w") as f:
    for (tabla, set_part, condiciones_fijas), lista_cla_trab in agrupaciones.items():
        valores_in = ", ".join(lista_cla_trab)
        select = f"SELECT * FROM {tabla} WHERE {condiciones_fijas} AND CLA_TRAB IN ({valores_in});"
        f.write(select + "\n")
        f.write("-- SET: " + set_part + "\n")
        f.write("\n")

print(f"Respaldo agrupado generado en: '{archivo_salida}' 🎉")


Respaldo agrupado generado en: 'backup_agrupado.sql' 🎉


In [ ]:
import re
from collections import defaultdict

archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# Leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir en sentencias por punto y coma
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Estructura para agrupar:
# clave: (tabla, set, CLA_EMPRESA) --> valores: lista de tuples (CLA_TRAB, NUM_PLAZA)
agrupaciones = defaultdict(list)

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    if sentencia.upper().startswith("UPDATE"):
        # Tabla
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        tabla = tabla_match.group(1).strip() if tabla_match else "UNKNOWN"

        # SET
        set_match = re.search(r"SET\s+(.*?)\s+WHERE", sentencia, re.IGNORECASE | re.DOTALL)
        set_part = set_match.group(1).strip() if set_match else "UNKNOWN"

        # WHERE
        where_match = re.search(r"WHERE\s+(.*)", sentencia, re.IGNORECASE | re.DOTALL)
        where_part = where_match.group(1).strip() if where_match else ""

        # Separar condiciones
        condiciones = [cond.strip() for cond in re.split(r"\s+AND\s+", where_part, flags=re.IGNORECASE)]

        # Inicializar
        cla_empresa = None
        cla_trab = None
        num_plaza = None

        for cond in condiciones:
            if cond.upper().startswith("CLA_EMPRESA"):
                match = re.search(r"CLA_EMPRESA\s*=\s*(\d+)", cond, re.IGNORECASE)
                if match:
                    cla_empresa = match.group(1)
            elif cond.upper().startswith("CLA_TRAB"):
                match = re.search(r"CLA_TRAB\s*=\s*(\d+)", cond, re.IGNORECASE)
                if match:
                    cla_trab = match.group(1)
            elif cond.upper().startswith("NUM_PLAZA"):
                match = re.search(r"NUM_PLAZA\s*=\s*'([^']*)'", cond, re.IGNORECASE)
                if match:
                    num_plaza = match.group(1)

        # Usar clave fija
        clave = (tabla, set_part, cla_empresa)
        agrupaciones[clave].append( (cla_trab, num_plaza) )

# Generar SELECTs agrupados
with open(archivo_salida, "w") as f:
    for (tabla, set_part, cla_empresa), lista in agrupaciones.items():
        # Listas únicas de valores
        lista_cla_trab = sorted(set(item[0] for item in lista))
        lista_num_plaza = sorted(set(item[1] for item in lista))

        in_trab = ", ".join(lista_cla_trab)
        in_plaza = ", ".join(f"'{p}'" for p in lista_num_plaza)

        select = f"SELECT * FROM {tabla} WHERE CLA_EMPRESA = {cla_empresa} AND NUM_PLAZA IN ({in_plaza}) AND CLA_TRAB IN ({in_trab});"
        f.write(select + "\n")
        f.write("-- SET: " + set_part + "\n")
        f.write("\n")

print(f"Respaldo agrupado final generado en: '{archivo_salida}' 🚀")


Respaldo agrupado final generado en: 'backup_agrupado.sql' 🚀


In [ ]:
import re
from collections import defaultdict

# 🗂️ Archivos
archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# 📥 Leer archivo completo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# 🔍 Dividir en sentencias (por ; y salto de línea opcional)
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# 💾 Agrupamos por clave: (tabla, columna) ➜ lista de valores
condiciones_agrupadas = defaultdict(set)

# 🔥 Aquí guardaremos también los SELECTs directos de los UPDATE
selects_update = []

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    sentencia_upper = sentencia.upper()

    # 🗑️ DELETE
    if sentencia_upper.startswith("DELETE FROM"):
        # Extraer tabla
        tabla_match = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # Extraer WHERE
            where_idx = sentencia_upper.find("WHERE")
            if where_idx != -1:
                condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
                # Dividir por AND
                condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]
                for cond in condiciones:
                    match = re.match(r"(\w+)\s*=\s*(.+)", cond)
                    if match:
                        columna = match.group(1).strip()
                        valor = match.group(2).strip()
                        condiciones_agrupadas[(tabla, columna)].add(valor)

    # ✏️ UPDATE
    elif sentencia_upper.startswith("UPDATE"):
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            where_idx = sentencia_upper.find("WHERE")
            if where_idx != -1:
                condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
                # Dividir por AND para identificar condiciones
                condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]
                for cond in condiciones:
                    match = re.match(r"(\w+)\s*=\s*(.+)", cond)
                    if match:
                        columna = match.group(1).strip()
                        valor = match.group(2).strip()
                        condiciones_agrupadas[(tabla, columna)].add(valor)
                # También generamos el SELECT directo para respaldo
                select_line = f"SELECT * FROM {tabla} WHERE {condiciones_raw};"
                selects_update.append(select_line)

# ✅ Generamos las sentencias SELECT de respaldo agrupadas
sentencias_finales = []

for (tabla, columna), valores in condiciones_agrupadas.items():
    if valores:
        valores_formateados = ", ".join(sorted(valores))
        select = f"SELECT * FROM {tabla} WHERE {columna} IN ({valores_formateados});"
        sentencias_finales.append(select)

# Añadimos los SELECT directos de UPDATE (en caso de quererlos todos juntos)
sentencias_finales.extend(selects_update)

# 💾 Guardamos el resultado final
with open(archivo_salida, "w") as f:
    for sel in sentencias_finales:
        f.write(sel + "\n")

print(f"✅ {len(sentencias_finales)} sentencias de respaldo generadas en '{archivo_salida}' 🚀🎉")


✅ 20 sentencias de respaldo generadas en 'backup_agrupado.sql' 🚀🎉


In [ ]:
import re
from collections import defaultdict

# Archivos
archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# Leer archivo completo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir por ;
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Estructura:
# {(tabla, columnas_tuple): {columna: set(valores)}}
agrupaciones = {}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    sentencia_upper = sentencia.upper()

    # Identificar tabla (DELETE o UPDATE)
    tabla = None
    if sentencia_upper.startswith("DELETE FROM"):
        match_tabla = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
    elif sentencia_upper.startswith("UPDATE"):
        match_tabla = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
    else:
        continue

    if not match_tabla:
        continue

    tabla = match_tabla.group(1)

    # Obtener el WHERE completo
    where_idx = sentencia_upper.find("WHERE")
    if where_idx == -1:
        continue

    condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
    # Separar por AND
    condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]

    # Dict para esta combinación de columnas
    columnas = []
    valores_por_columna = defaultdict(set)

    for cond in condiciones:
        match = re.match(r"(\w+)\s*=\s*(.+)", cond)
        if match:
            columna = match.group(1).strip()
            valor = match.group(2).strip()
            columnas.append(columna)
            valores_por_columna[columna].add(valor)

    # Ordenar columnas para que siempre sea la misma clave (sin importar orden en SQL)
    columnas_tuple = tuple(sorted(columnas))

    clave = (tabla, columnas_tuple)
    if clave not in agrupaciones:
        agrupaciones[clave] = defaultdict(set)

    for columna, valores in valores_por_columna.items():
        agrupaciones[clave][columna].update(valores)

# Generar SELECTs agrupados finales
with open(archivo_salida, "w") as f:
    for (tabla, columnas), valores_por_columna in agrupaciones.items():
        condiciones = []
        for columna in columnas:
            valores = valores_por_columna[columna]
            # Formatear lista de valores (cuidando strings con comillas)
            valores_formateados = ", ".join(sorted(valores))
            condicion = f"{columna} IN ({valores_formateados})"
            condiciones.append(condicion)
        where_completo = " AND ".join(condiciones)
        select_final = f"SELECT * FROM {tabla} WHERE {where_completo};"
        f.write(select_final + "\n")

print(f"✅ {len(agrupaciones)} sentencias SELECT agrupadas generadas en '{archivo_salida}' 🚀🎉")


✅ 1 sentencias SELECT agrupadas generadas en 'backup_agrupado.sql' 🚀🎉


In [ ]:
import re
from collections import defaultdict

# Archivos
archivo_entrada = "/content/entrada.sql"
archivo_salida = "backup_agrupado.sql"

# Leer archivo completo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir por ;
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Estructura:
# {(tabla, columnas_tuple): {columna: set(valores)}}
agrupaciones = {}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    sentencia_upper = sentencia.upper()

    # Identificar tabla (DELETE o UPDATE)
    tabla = None
    if sentencia_upper.startswith("DELETE FROM"):
        match_tabla = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
    elif sentencia_upper.startswith("UPDATE"):
        match_tabla = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
    else:
        continue

    if not match_tabla:
        continue

    tabla = match_tabla.group(1)

    # Obtener el WHERE completo
    where_idx = sentencia_upper.find("WHERE")
    if where_idx == -1:
        continue

    condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
    # Separar por AND
    condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]

    # Dict para esta combinación de columnas
    columnas = []
    valores_por_columna = defaultdict(set)

    for cond in condiciones:
        # Buscar "columna = valor"
        match_eq = re.match(r"(\w+)\s*=\s*(.+)", cond)
        if match_eq:
            columna = match_eq.group(1).strip()
            valor = match_eq.group(2).strip()
            columnas.append(columna)
            valores_por_columna[columna].add(valor)
            continue

        # Buscar "columna IN (valores)"
        match_in = re.match(r"(\w+)\s+IN\s*\((.+)\)", cond, re.IGNORECASE)
        if match_in:
            columna = match_in.group(1).strip()
            valores_raw = match_in.group(2).strip()
            valores = [v.strip() for v in valores_raw.split(",")]
            columnas.append(columna)
            for valor in valores:
                valores_por_columna[columna].add(valor)
            continue

    # Ordenar columnas para clave única
    columnas_tuple = tuple(sorted(columnas))
    clave = (tabla, columnas_tuple)
    if clave not in agrupaciones:
        agrupaciones[clave] = defaultdict(set)

    for columna, valores in valores_por_columna.items():
        agrupaciones[clave][columna].update(valores)

# Generar SELECTs agrupados finales
with open(archivo_salida, "w") as f:
    for (tabla, columnas), valores_por_columna in agrupaciones.items():
        condiciones = []
        for columna in columnas:
            valores = valores_por_columna[columna]
            valores_formateados = ", ".join(valores)
            condicion = f"{columna} IN ({valores_formateados})"
            condiciones.append(condicion)
        where_completo = " AND ".join(condiciones)
        select_final = f"SELECT * FROM {tabla} WHERE {where_completo};"
        f.write(select_final + "\n")

print(f"Correcto: {len(agrupaciones)} sentencias SELECT generadas en '{archivo_salida}'")


Correcto: 1 sentencias SELECT generadas en 'backup_agrupado.sql'


In [ ]:
import re
from collections import defaultdict

#1. Generar nombre de archivo backup.
print("🔵 Nombre del backup. 🔵")
print("Backup_[Tipo]_[Número]_[TareaYNúmero]_Parte[Número]_[Descripción].[Extensión]\n")

#Tipo.
print("🌐 Ingresa el tipo:")
print("1. Ticket.")
print("2. RFC.")
while True:
    tipo_opcion = input("▶ Opción: ").strip()
    if tipo_opcion in ("1", "2"):
        tipo = "Ticket" if tipo_opcion == "1" else "RFC"
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

#Número.
print(f"\n🌐 Ingresa el número de {tipo}:")
numero = input(f"▶ Número de {tipo}: ").strip()
while not numero:
    print("⚠️ Obligatorio.")
    numero = input(f"▶ Número de {tipo}: ").strip()

#Tarea.
print("\n🌐 Ingresa la tarea:")
print("1. Inicial1.")
print("2. Complementaria.")
while True:
    tarea_opcion = input("▶ Opción: ").strip()
    if tarea_opcion == "1":
        tarea = "Inicial1"
        break
    elif tarea_opcion == "2":
        tarea_num = input("▶ Número de complementaria: ").strip()
        while not tarea_num.isdigit():
            print("⚠️ Debe ser un número.")
            tarea_num = input("▶ Número de complementaria: ").strip()
        tarea = f"Complementaria{tarea_num}"
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

#Parte.
print(f"\n🌐 Ingresa la parte de la {tarea}:")
parte = input(f"▶ Parte: ").strip()
while not parte.isdigit():
    print("⚠️ Debe ser un número.")
    parte = input(f"▶ Parte: ").strip()

#Descripción.
print("\n🌐 Agregar una descripción:")
descripcion = input("▶ Descripción (opcional): ").strip()

# 👉 Extensión
print("\n🌐 ¿El archivo tendrá extensión .sql?")
print("1. Sí")
print("2. No")
while True:
    extension_opcion = input("▶ Opción: ").strip()
    if extension_opcion == "1":
        extension = "sql"
        break
    elif extension_opcion == "2":
        extension = input("▶ Ingresa la extensión deseada (ejemplo txt, csv): ").strip().lower()
        while not extension:
            print("⚠️ Obligatorio.")
            extension = input("▶ Extensión deseada: ").strip().lower()
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

# Generar nombre final
nombre_archivo = f"Backup_{tipo}_{numero}_{tarea}_Parte{parte}"
if descripcion:
    nombre_archivo += f"_{descripcion}"
nombre_archivo += f".{extension}"

#print(f"\nNombre de archivo final generado: {nombre_archivo}\n")

#---------------------------------------------------------------------------------

#2. Procesar archivo SQL de entrada y agrupar.
archivo_entrada = "/content/entrada.sql"
archivo_salida = nombre_archivo

with open(archivo_entrada, "r") as f:
    contenido = f.read()

sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)
agrupaciones = {}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    sentencia_upper = sentencia.upper()
    tabla = None
    if sentencia_upper.startswith("DELETE FROM"):
        match_tabla = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
    elif sentencia_upper.startswith("UPDATE"):
        match_tabla = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
    else:
        continue

    if not match_tabla:
        continue

    tabla = match_tabla.group(1)
    where_idx = sentencia_upper.find("WHERE")
    if where_idx == -1:
        continue

    condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
    condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]
    columnas = []
    valores_por_columna = defaultdict(set)

    for cond in condiciones:
        match_eq = re.match(r"(\w+)\s*=\s*(.+)", cond)
        if match_eq:
            columna = match_eq.group(1).strip()
            valor = match_eq.group(2).strip()
            columnas.append(columna)
            valores_por_columna[columna].add(valor)
            continue

        match_in = re.match(r"(\w+)\s+IN\s*\((.+)\)", cond, re.IGNORECASE)
        if match_in:
            columna = match_in.group(1).strip()
            valores_raw = match_in.group(2).strip()
            valores = [v.strip() for v in valores_raw.split(",")]
            columnas.append(columna)
            for valor in valores:
                valores_por_columna[columna].add(valor)
            continue

    columnas_tuple = tuple(sorted(columnas))
    clave = (tabla, columnas_tuple)
    if clave not in agrupaciones:
        agrupaciones[clave] = defaultdict(set)
    for columna, valores in valores_por_columna.items():
        agrupaciones[clave][columna].update(valores)

#---------------------------------------------------------------------------------

#3. Generar y guardar archivo final
with open(archivo_salida, "w") as f:
    for (tabla, columnas), valores_por_columna in agrupaciones.items():
        condiciones = []
        for columna in columnas:
            valores = valores_por_columna[columna]
            valores_formateados = ", ".join(valores)
            condiciones.append(f"{columna} IN ({valores_formateados})")
        where_final = " AND ".join(condiciones)
        select = f"SELECT * FROM {tabla} WHERE {where_final};"
        f.write(select + "\n")

print(f"\n\n✅ Archivo '{archivo_salida}' generado exitosamente con {len(agrupaciones)} SELECTs.")

🔵 Nombre del backup. 🔵
Backup_[Tipo]_[Número]_[TareaYNúmero]_Parte[Número]_[Descripción].[Extensión]

🌐 Ingresa el tipo:
1. Ticket.
2. RFC.
▶ Opción: 1

🌐 Ingresa el número de Ticket:
▶ Número de Ticket: 33749

🌐 Ingresa la tarea:
1. Inicial1.
2. Complementaria.
▶ Opción: 1

🌐 Ingresa la parte de la Inicial1:
▶ Parte: 25

🌐 Agregar una descripción:
▶ Descripción (opcional): 

🌐 ¿El archivo tendrá extensión .sql?
1. Sí
2. No
▶ Opción: |
⚠️ Opción inválida. Ingresa 1 o 2.
▶ Opción: 1

\✅ Archivo 'Backup_Ticket_33749_Inicial1_Parte25.sql' generado exitosamente con 1 SELECTs.


In [1]:
import os
import re
from collections import defaultdict

archivo_entrada = "/content/entrada.sql"

#Validar que exista el archivo antes de continuar.
if not os.path.exists(archivo_entrada):
    print("Archivo no encontrado. Cargar el archivo.")
    input("Pulsa ENTER para cerrar.")
    exit() #No efectua cambios en Google Collab.

#1. Generar nombre de archivo backup.
print("🔵 Nombre del backup. 🔵")
#print("Backup_[Tipo]_[Número]_[TareaYNúmero]_Parte[Número]_[Descripción].[Extensión]\n")

#Tipo.
print("🌐 Ingresa el tipo:")
print("1. Ticket.")
print("2. RFC.")
while True:
    tipo_opcion = input("▶ Opción: ").strip()
    if tipo_opcion in ("1", "2"):
        tipo = "Ticket" if tipo_opcion == "1" else "RFC"
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

#Número.
print(f"\n🌐 Ingresa el número de {tipo}:")
numero = input(f"▶ Número de {tipo}: ").strip()
while not numero:
    print("⚠️ Obligatorio.")
    numero = input(f"▶ Número de {tipo}: ").strip()

#Tarea.
print("\n🌐 Ingresa la tarea:")
print("1. Inicial1.")
print("2. Complementaria.")
while True:
    tarea_opcion = input("▶ Opción: ").strip()
    if tarea_opcion == "1":
        tarea = "Inicial1"
        break
    elif tarea_opcion == "2":
        tarea_num = input("▶ Número de complementaria: ").strip()
        while not tarea_num.isdigit():
            print("⚠️ Debe ser un número.")
            tarea_num = input("▶ Número de complementaria: ").strip()
        tarea = f"Complementaria{tarea_num}"
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

#Parte.
print(f"\n🌐 Ingresa la parte de la {tarea}:")
parte = input(f"▶ Parte: ").strip()
while not parte.isdigit():
    print("⚠️ Debe ser un número.")
    parte = input(f"▶ Parte: ").strip()

#Descripción.
print("\n🌐 Agregar una descripción:")
descripcion = input("▶ Descripción (opcional): ").strip()

#Extensión.
print("\n🌐 ¿El archivo tendrá extensión .sql?")
print("1. Sí")
print("2. No")
while True:
    extension_opcion = input("▶ Opción: ").strip()
    if extension_opcion == "1":
        extension = "sql"
        break
    elif extension_opcion == "2":
        extension = input("▶ Ingresa la extensión deseada (ejemplo txt, csv): ").strip().lower()
        while not extension:
            print("⚠️ Obligatorio.")
            extension = input("▶ Extensión deseada: ").strip().lower()
        break
    else:
        print("⚠️ Opción inválida. Ingresa 1 o 2.")

# Generar nombre final
nombre_archivo = f"Backup_{tipo}_{numero}_{tarea}_Parte{parte}"
if descripcion:
    nombre_archivo += f"_{descripcion}"
nombre_archivo += f".{extension}"

#print(f"\nNombre de archivo final generado: {nombre_archivo}\n")

# ---------------------------------------------------------------------------------

#2. Procesar archivo SQL de entrada y agrupar.
archivo_entrada = "/content/entrada.sql"
archivo_salida = nombre_archivo

with open(archivo_entrada, "r") as f:
    contenido = f.read()

sentencias = contenido.splitlines()
agrupaciones = {}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    if not sentencia or sentencia.startswith("--"):
        continue

    sentencia_upper = sentencia.upper()
    tabla = None
    if sentencia_upper.startswith("DELETE FROM"):
        match_tabla = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
    elif sentencia_upper.startswith("UPDATE"):
        match_tabla = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
    else:
        continue

    if not match_tabla:
        continue

    tabla = match_tabla.group(1)
    where_idx = sentencia_upper.find("WHERE")
    if where_idx == -1:
        continue

    condiciones_raw = sentencia[where_idx + len("WHERE"):].strip()
    condiciones = [c.strip() for c in re.split(r"\s+AND\s+", condiciones_raw, flags=re.IGNORECASE)]
    columnas = []
    valores_por_columna = defaultdict(set)

    for cond in condiciones:
        match_eq = re.match(r"(\w+)\s*=\s*(.+)", cond)
        if match_eq:
            columna = match_eq.group(1).strip()
            valor = match_eq.group(2).strip()
            columnas.append(columna)
            valores_por_columna[columna].add(valor)
            continue

        match_in = re.match(r"(\w+)\s+IN\s*\((.+)\)", cond, re.IGNORECASE)
        if match_in:
            columna = match_in.group(1).strip()
            valores_raw = match_in.group(2).strip()
            valores = [v.strip() for v in valores_raw.split(",")]
            columnas.append(columna)
            for valor in valores:
                valores_por_columna[columna].add(valor)
            continue

    columnas_tuple = tuple(sorted(columnas))
    clave = (tabla, columnas_tuple)
    if clave not in agrupaciones:
        agrupaciones[clave] = defaultdict(set)
    for columna, valores in valores_por_columna.items():
        agrupaciones[clave][columna].update(valores)

# ---------------------------------------------------------------------------------

#3. Generar y guardar archivo final.
with open(archivo_salida, "w") as f:
    for (tabla, columnas), valores_por_columna in agrupaciones.items():
        condiciones = []
        for columna in columnas:
            valores = valores_por_columna[columna]
            valores_formateados = ", ".join(valores)
            condiciones.append(f"{columna} IN ({valores_formateados})")
        where_final = " AND ".join(condiciones)
        select = f"SELECT * FROM {tabla} WHERE {where_final};"
        f.write(select + "\n")

print(f"\n\n✅ Archivo '{archivo_salida}' generado exitosamente con {len(agrupaciones)} SELECTs.")


Archivo no encontrado. Cargar el archivo.
Pulsa ENTER para cerrar.
🔵 Nombre del backup. 🔵
🌐 Ingresa el tipo:
1. Ticket.
2. RFC.


KeyboardInterrupt: Interrupted by user